In [4]:
import pandas as pd
import ccxt

## Data obtained from CoinDesk 
------------------------------
### freq: daily 
### informations: Closing Price(USD), 24h Open(USD), 24h High(USD), 24h Low(USD)

In [2]:
btc = pd.read_csv("../data/0_raw/BTC_USD_2013-10-01_2021-04-21-CoinDesk.csv")

In [21]:
btc

,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
0,BTC,2013-10-01,123.654990,124.304660,124.751660,122.563490
1,BTC,2013-10-02,125.455000,123.654990,125.758500,123.633830
2,BTC,2013-10-03,108.584830,125.455000,125.665660,83.328330
3,BTC,2013-10-04,118.674660,108.584830,118.675000,107.058160
4,BTC,2013-10-05,121.338660,118.674660,121.936330,118.005660
...,...,...,...,...,...,...
2754,BTC,2021-04-17,61965.782598,63225.093917,63520.325374,60033.534667
2755,BTC,2021-04-18,60574.444728,61444.232503,62534.028498,59802.889267
2756,BTC,2021-04-19,56850.830166,60191.525406,60531.988848,52148.983544
2757,BTC,2021-04-20,56224.101588,56335.389141,57609.368118,54449.245330


In [33]:
eth = pd.read_csv("../data/0_raw/ETH_USD_2015-08-09_2021-04-21-CoinDesk.csv")

In [34]:
eth

,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
0,ETH,2015-08-09,0.909046,1.749289,1.916540,0.794497
1,ETH,2015-08-10,0.692321,0.909046,0.909046,0.692321
2,ETH,2015-08-11,0.668067,0.692321,0.692321,0.654331
3,ETH,2015-08-12,0.850151,0.668067,1.148621,0.668067
4,ETH,2015-08-13,1.266023,0.850151,1.266023,0.850151
...,...,...,...,...,...,...
2077,ETH,2021-04-17,2438.429594,2516.393168,2541.548966,2328.830718
2078,ETH,2021-04-18,2374.755810,2426.476182,2495.921609,2320.285017
2079,ETH,2021-04-19,2249.008696,2323.384415,2344.369598,1978.224247
2080,ETH,2021-04-20,2201.648351,2242.321493,2278.921376,2086.202945


## Data obtaining from ccxt library 
--------------------------------------------
motivation, get high frequency data from exchange
exchange(s) should be decided \
Another thing is that cryptocurrency against which currency should be decided. \
anlamadığım bir durum var doları cryptocurrency -lere karşı bulamıyorum

In [ ]:
#https://backtest-rookies.com/2018/03/08/download-cryptocurrency-data-with-ccxt/
#https://www.linkedin.com/pulse/download-historical-data-all-cryptocoins-ccxt-gursel-karacor/?articleId=6636391574811090944   
## final: https://github.com/codeninja/CCXT-Historical-Data/blob/master/Binance%20Historical%20Data%20.ipynb

In [11]:
import os
from pathlib import Path

import sys
import csv

import ccxt
# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if len(ohlcv) == 0:
            break
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("../data/0_raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles+
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

In [4]:
scrape_candles_to_csv('./btc_busd_1d.csv', 'binance', 3, 'BTC/BUSD', '1d', '2019-01-0100:00:00Z', 1000)

597 BTC/BUSD candles in total from 2019-09-19T00:00:00.000Z to 2021-05-07T00:00:00.000Z
Saved 597 candles from 2019-09-19T00:00:00.000Z to 2021-05-07T00:00:00.000Z to ./btc_busd_1d.csv


In [6]:
scrape_candles_to_csv('./eth_busd_1d.csv', 'binance', 3, 'ETH/BUSD', '1d', '2019-01-0100:00:00Z', 1000)

565 ETH/BUSD candles in total from 2019-10-21T00:00:00.000Z to 2021-05-07T00:00:00.000Z
Saved 565 candles from 2019-10-21T00:00:00.000Z to 2021-05-07T00:00:00.000Z to ./eth_busd_1d.csv


In [17]:
scrape_candles_to_csv('./ltc_busd_1d.csv', 'binance', 3, 'LTC/BUSD', '1d', '2019-01-0100:00:00Z', 1000)

568 LTC/BUSD candles in total from 2019-10-21T00:00:00.000Z to 2021-05-10T00:00:00.000Z
Saved 568 candles from 2019-10-21T00:00:00.000Z to 2021-05-10T00:00:00.000Z to ./ltc_busd_1d.csv


In [ ]:
#BUSD ile çok az veri çekilebildiği için usdt yi de çektim.

In [25]:
scrape_candles_to_csv('./btc_usdt_1d.csv', 'binance', 3, 'BTC/USDT', '1d', '2017-01-0100:00:00Z', 1000)

1000 BTC/USDT candles in total from 2018-08-12T00:00:00.000Z to 2021-05-07T00:00:00.000Z
1360 BTC/USDT candles in total from 2017-08-17T00:00:00.000Z to 2021-05-07T00:00:00.000Z
Saved 1360 candles from 2017-08-17T00:00:00.000Z to 2021-05-07T00:00:00.000Z to ./btc_usdt_1d.csv


In [26]:
scrape_candles_to_csv('./eth_usdt_1d.csv', 'binance', 3, 'ETH/USDT', '1d', '2017-01-0100:00:00Z', 1000)

1000 ETH/USDT candles in total from 2018-08-12T00:00:00.000Z to 2021-05-07T00:00:00.000Z
1360 ETH/USDT candles in total from 2017-08-17T00:00:00.000Z to 2021-05-07T00:00:00.000Z
Saved 1360 candles from 2017-08-17T00:00:00.000Z to 2021-05-07T00:00:00.000Z to ./eth_usdt_1d.csv


In [14]:
scrape_candles_to_csv('./ltc_usdt_1d.csv', 'binance', 3, 'LTC/USDT', '1d', '2017-01-0100:00:00Z', 1000)

1000 LTC/USDT candles in total from 2018-08-15T00:00:00.000Z to 2021-05-10T00:00:00.000Z
1245 LTC/USDT candles in total from 2017-12-13T00:00:00.000Z to 2021-05-10T00:00:00.000Z
Saved 1245 candles from 2017-12-13T00:00:00.000Z to 2021-05-10T00:00:00.000Z to ./ltc_usdt_1d.csv


In [28]:
for filename in ["btc_busd_1d.csv", "eth_busd_1d.csv", "btc_usdt_1d.csv", "eth_usdt_1d.csv", ]:
    df = pd.read_csv('../data/0_raw/Binance/' + filename, header=None)
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Timestamp', inplace=True)
    df.index = df.index /1000
    df["Date"] = pd.to_datetime(df.index,unit='s')
    df.to_csv("../data/0_raw/" + filename)

In [18]:
for filename in ["ltc_busd_1d.csv", "ltc_usdt_1d.csv"]:
    df = pd.read_csv('../data/0_raw/Binance/' + filename, header=None)
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Timestamp', inplace=True)
    df.index = df.index /1000
    df["Date"] = pd.to_datetime(df.index,unit='s')
    df.to_csv("../data/0_raw/" + filename)

In [20]:
pd.read_csv("../data/0_raw/ltc_usdt_1d.csv", header =None)

,0,1,2,3,4,5,6
0,Timestamp,Open,High,Low,Close,Volume,Date
1,1513123200.0,272.0,330.0,260.0,290.01,9565.16019,2017-12-13
2,1513209600.0,290.01,302.72,252.0,272.4,9631.98382,2017-12-14
3,1513296000.0,272.4,314.21,239.99,294.0,16579.78235,2017-12-15
4,1513382400.0,294.0,305.0,285.0,293.97,9102.10005,2017-12-16
...,...,...,...,...,...,...,...
1241,1620259200.0,356.03,364.93,318.0,354.67,2752093.00755,2021-05-06
1242,1620345600.0,354.52,374.5,333.33,344.89,2822668.4659700003,2021-05-07
1243,1620432000.0,344.89,356.68,334.0,347.1,1327716.1907,2021-05-08
1244,1620518400.0,347.17,389.0,333.51,387.8,2292100.05852,2021-05-09


In [ ]:
scrape_candles_to_csv('./ltc_usdt_1d.csv', 'binance', 3, 'LTC/USDT', '1d', '2017-01-0100:00:00Z', 1000)